In [1]:
from utils import load_captions_txt

captions_dict = load_captions_txt("../data/Flickr8k_text/captions.txt")

all_captions = []
for caps in captions_dict.values():
    all_captions.extend(caps)

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(all_captions)

vocab_size = len(tokenizer.word_index) + 1

def max_caption_length(captions):
    return max(len(c.split()) for c in captions)

max_length = max_caption_length(all_captions)

print(f"Vocabulary size: {vocab_size}")
print(f"Max caption length: {max_length}")


Vocabulary size: 8497
Max caption length: 40


In [2]:
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.optimizers import Adam

with open("features/image_features_efficientnet.pkl", "rb") as f:
    image_features = pickle.load(f)

inputs1 = Input(shape=(1280,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 1280)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 40, 256)   │  2,175,232 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1280)      │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 40, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 40)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    327,936 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8497)      │  2,183,729 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,278,001 (20.13 MB)

 Trainable params: 5,278,001 (20.13 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
from utils import data_generator_single_example

In [5]:
import tensorflow as tf
import functools

batch_size = 64

# --- MODIFICA QUESTA RIGA ---
# La signature deve corrispondere alla nuova dimensione delle feature (1280)
output_signature_single = (
    (
        tf.TensorSpec(shape=(1280,), dtype=tf.float32), # <-- CAMBIA 2048 IN 1280       
        tf.TensorSpec(shape=(max_length,), dtype=tf.int32)      
    ),
    tf.TensorSpec(shape=(vocab_size,), dtype=tf.float32)         
)
# -----------------------------

# Il resto del codice della cella non cambia
partial_generator = functools.partial(
    data_generator_single_example,
    captions_dict, 
    image_features, 
    tokenizer, 
    max_length, 
    vocab_size
)

dataset = tf.data.Dataset.from_generator(
    partial_generator,
    output_signature=output_signature_single # Ora la signature è corretta
)

dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

steps_per_epoch = len(captions_dict) // batch_size

# Questo codice di verifica ora funzionerà
print("Verifica del primo batch dal dataset...")
for batch in dataset.take(1):
    (X1, X2), y = batch
    print("\n--- BATCH RICEVUTO CORRETTAMENTE DA TENSORFLOW ---")
    print("X1 shape:", X1.shape) # Dovrebbe stampare (64, 1280)
    print("X2 shape:", X2.shape)
    print("y shape:", y.shape)

Verifica del primo batch dal dataset...

--- BATCH RICEVUTO CORRETTAMENTE DA TENSORFLOW ---
X1 shape: (64, 1280)
X2 shape: (64, 40)
y shape: (64, 8497)


In [6]:
steps = len(captions_dict) // batch_size
#-----------------------------

print(f"Inizio addestramento con {steps} steps per epoca.")

model.fit(
    dataset,
    epochs=1,
    steps_per_epoch=steps, 
    verbose=1
)


import os
print("Addestramento completato. Salvataggio del modello...")
save_dir = "models"
save_path = os.path.join(save_dir, "model_caption:double.keras")

# --- AGGIUNGI QUESTA RIGA FONDAMENTALE ---
# Crea la directory di salvataggio se non esiste già.
# exist_ok=True evita un errore se la cartella esiste già.
os.makedirs(save_dir, exist_ok=True)
# ----------------------------------------

# Salva il modello nel nuovo e più robusto formato
print(f"Addestramento completato. Salvataggio del modello in: {save_path}")
model.save(save_path)

# Il salvataggio del tokenizer non cambia
tokenizer_path = 'tokenizer.json' # Salva nella cartella principale per semplicità
tokenizer_json = tokenizer.to_json()
with open(tokenizer_path, 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

print(f"✔️ Modello e tokenizer salvati correttamente.")

Inizio addestramento con 126 steps per epoca.
126/126 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - accuracy: 0.1238 - loss: 6.7276
Addestramento completato. Salvataggio del modello...
Addestramento completato. Salvataggio del modello in: models\model_caption:double.keras
✔️ Modello e tokenizer salvati correttamente.
